Notebook Entrainement de modèle depuis les données mongoDB vers MLFLOW

In [1]:
import os
import json
import pymongo
import mlflow
import mlflow.sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from pyspark.sql.types import StructType, StructField, StringType, MapType, TimestampType
from transformers import DistilBertTokenizer
from transformers import DistilBertModel
import torch
import torch.nn as nn

/opt/bitnami/python/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType

# Define a UDF to convert the string representation of the array to an actual array of floats
def parse_array(col):
    return [float(x) for x in col.strip('[]').split(', ')]

parse_array_udf = udf(parse_array, ArrayType(FloatType()))


# Create a Spark session with the MongoDB Spark Connector package
spark = SparkSession.builder \
    .appName("myApp") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()


schema = StructType([
    StructField("user", StringType(), True),
    StructField("repo", StringType(), True),
    StructField("mainLanguage", StringType(), True),
    StructField("languages", MapType(StringType(), StringType()), True),
    StructField("readme", StringType(), True),
    StructField("processed_readme", StringType(), True),
    StructField("last_updated", TimestampType(), True),
])

df = spark.read \
    .format("mongo") \
    .option("database", "dev") \
    .option("collection", "raw_data") \
    .option("uri", "mongodb://mongo:27017/") \
    .schema(schema) \
    .load()

df = df.withColumn("processed_readme", parse_array_udf(df["processed_readme"]))

df.show()

+--------------+--------------------+------------+--------------------+------------------------+--------------------+--------------------+
|          user|                repo|mainLanguage|           languages|                  readme|    processed_readme|        last_updated|
+--------------+--------------------+------------+--------------------+------------------------+--------------------+--------------------+
|     citusdata|               citus|           C|{sed -> NULL, She...|    | **<br/>The Citu...|[-0.18333645, -0....|2025-03-06 14:33:...|
|         apple|          darwin-xnu|           C|{Roff -> NULL, Aw...|    What is XNU?\n===...|[-0.5111101, -0.1...|2025-03-06 14:33:...|
|       openzfs|                 zfs|           C|{Roff -> NULL, se...|    ![img](https://op...|[-0.3444045, -0.1...|2025-03-06 14:33:...|
|           h2o|                 h2o|           C|{JavaScript -> NU...|    H2O - an optimize...|[-0.18877514, -0....|2025-03-06 14:33:...|
|      libretro|           

In [39]:
num_lines = df.count()
print(f"Number of lines in the dataframe: {num_lines}")

Number of lines in the dataframe: 527


In [44]:
# Charger les données depuis MongoDB
def load_data_from_mongo():
    data = df.select("processed_readme", "mainLanguage").collect()
    texts = [row["processed_readme"] for row in data]
    labels = [row["mainLanguage"] for row in data]
    return texts, labels


In [45]:
# Charger le tokenizer et le modèle DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
bert_model = DistilBertModel.from_pretrained("distilbert-base-uncased")

def encode_texts_with_bert(texts, tokenizer, model):
    """Tokenise et vectorise les textes avec DistilBERT."""
    encoded_texts = []
    
    for text in texts:
        inputs = tokenizer(text, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        with torch.no_grad():  # Pas besoin de calculer les gradients
            outputs = model(**inputs)
        
        # On prend le CLS token ([0, 0, :]) qui représente l'ensemble du texte
        sentence_embedding = outputs.last_hidden_state[:, 0, :].squeeze().numpy()
        encoded_texts.append(sentence_embedding)
    
    return np.array(encoded_texts)

# Charger et préparer les données
texts, labels = load_data_from_mongo()
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)



In [46]:
# Transformer les textes en embeddings DistilBERT
#document_vectors = encode_texts_with_bert(texts, tokenizer, bert_model)

# Division des données en train/test
#X_train, X_test, y_train, y_test = train_test_split(document_vectors, encoded_labels, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(texts, encoded_labels, test_size=0.2, random_state=42)


In [47]:
#Entraînement du classifieur
classifier = RandomForestClassifier(n_estimators=100, random_state=42)
classifier.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [48]:
#Évaluation du modèle
train_score = classifier.score(X_train, y_train)
test_score = classifier.score(X_test, y_test)

print(f"Train Accuracy: {train_score:.4f}")
print(f"Test Accuracy: {test_score:.4f}")



Train Accuracy: 1.0000
Test Accuracy: 0.4554


In [49]:
#Enregistrement dans MLflow
mlflow.set_tracking_uri("http://mlflow:8080")

with mlflow.start_run():
    mlflow.sklearn.log_model(classifier, "random_forest_model")
    mlflow.log_param("model_type", "RandomForestClassifier")
    mlflow.log_metric("train_accuracy", train_score)
    mlflow.log_metric("test_accuracy", test_score)

print("Modèle entraîné et enregistré dans MLflow")

2025/03/06 14:43:44 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2025/03/06 14:44:18 WARNING mlflow.models.model: Model logged without a signature and input example. Ple

🏃 View run useful-gnu-213 at: http://mlflow:8080/#/experiments/0/runs/ab09f42063b64a05b70297a3e94267a7
🧪 View experiment at: http://mlflow:8080/#/experiments/0
Modèle entraîné et enregistré dans MLflow
